# ユーザーグループを設定します

ユーザーのグループを設定します。

所属グループに応じたリソース制限設定に使用します。

# グループの定義

グループ名

In [ ]:
group_name = 'group1'
group_name

このグループから削除するユーザーのメールアドレス

In [ ]:
user_email_list = '''
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''

In [ ]:
import re
user_emails = user_email_list.splitlines()
user_emails = [x.strip() for x in user_emails 
               if re.sub(r'#.*$', '', x).strip()]
user_emails

In [ ]:
import hashlib

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

user_names = [get_username_from_mail_address(x) for x in user_emails]
user_names

# グループの設定

[API Tokens](/hub/token)のページで、tokenを作成して以下に入力してください。

In [ ]:
import getpass
apitoken = getpass.getpass('JupyterHub API token')

In [ ]:
import os
api_url = os.environ['JUPYTERHUB_API_URL']

グループの存在確認

In [ ]:
headers = {
    'Authorization': 'token %s' % apitoken,
}

In [ ]:
import requests
import json

r = requests.get(api_url + '/groups/{}'.format(group_name),
                 headers=headers)
assert r.status_code == 200

group = json.loads(r.text)
group

In [ ]:
import json

r = requests.get(api_url + '/groups/{}'.format(group_name),
                 headers=headers)
group = json.loads(r.text)
current_users = group['users']

user2mail = dict(zip(user_names, user_emails))

nonmember_users = set(user_names) - set(current_users)
remove_users = set(user_names) - set(nonmember_users)

if remove_users:
    print("Remove the following users from group '{}'".format(group_name))
    for username in remove_users:
        print('{}({})'.format(user2mail[username], username))
    h = headers.copy()
    h['Content-Type'] : 'application/json'
    r = requests.delete(api_url + '/groups/{}/users'.format(group_name),
                      data = json.dumps({
                          "users" : list(remove_users)
                      }),
                      headers=h)
    assert r.status_code == 200

nonmember_users = set(user_names) - set(remove_users)
if nonmember_users:
    print("The fowllowing users are not member of group '{}'".format(group_name))
    for username in nonmember_users:
        print('{}({})'.format(user2mail[username], username))


確認

In [ ]:
r = requests.get(api_url + '/groups/{}'.format(group_name),
                  headers=headers)
group = json.loads(r.text)
group